<a href="https://colab.research.google.com/github/osmarbraz/exemplos_Llama/blob/main/ExemplosGeracaoTexto_Llama1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Exemplo de Geração de textos usando Llama v1.0 usando Transformers by HuggingFace


Exemplo de uso do modelo de linguagem grande Llama v1.0.
- Análise da geração de textos
- Prompts com textos emparelhados
- Injentando padrões no prompt
- Padrão Persona

Pré-requisitos:
- Configurar o notebook para usar GPU- Acesse o menu 'Ambiente de Execução -> Alterar o tipo do ambiente de execução -> Acelerador de hardware -> T4 GPU


**Notebook de referência:**

https://github.com/guardiaum/tutorial-sbbd2023/blob/main/Prompt_Engineering.ipynb


**Lista dos modelos:**

https://huggingface.co/models

**Link biblioteca Huggingface:**

https://github.com/huggingface/transformers




# 0 - Preparação do ambiente
Preparação do ambiente para execução do exemplo.

## Tratamento de logs

Método para tratamento dos logs.

In [ ]:
# Biblioteca de logging
import logging

# Formatando a mensagem de logging
logging.basicConfig(format="%(asctime)s : %(levelname)s : %(message)s", level=logging.INFO)

## Identificando o ambiente Colab

Cria uma variável para identificar que o notebook está sendo executado no Google Colaboratory.

In [ ]:
# Se estiver executando no Google Colaboratory
import sys

# Retorna true ou false se estiver no Google Colaboratory
IN_COLAB = "google.colab" in sys.modules

Função auxiliar para formatar o tempo como `hh: mm: ss`

In [ ]:
# Import das bibliotecas.
import time
import datetime

def formataTempo(tempo):
    """
      Pega a tempo em segundos e retorna uma string hh:mm:ss
    """
    # Arredonda para o segundo mais próximo.
    tempo_arredondado = int(round((tempo)))

    # Formata como hh:mm:ss
    return str(datetime.timedelta(seconds=tempo_arredondado))

# 1 - Instalação das bibliotecas

PEFT, ou Parameter-Efficient Fine-Tuning (PEFT), é uma biblioteca para adaptar com eficiência modelos de linguagem pré-treinados (PLMs) a vários aplicativos downstream sem ajustar todos os parâmetros do modelo. Os métodos PEFT ajustam apenas um pequeno número de parâmetros (extras) do modelo, diminuindo significativamente os custos computacionais e de armazenamento porque o ajuste fino de PLMs em grande escala é proibitivamente caro. Técnicas PEFT de última geração alcançam desempenho comparável ao do ajuste fino completo.

In [ ]:
!pip install peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.6/85.6 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 35.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.0/261.0 kB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 63.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 37.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 52.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 36.8 MB/s eta 0:00:00


Dependência do LlamaTokenizer

In [ ]:
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 11.5 MB/s eta 0:00:00


O bitsandbytes é um wrapper leve em torno de funções personalizadas CUDA, em particular otimizadores de 8 bits, multiplicação de matrizes (LLM.int8()) e funções de quantização. É uma dependência do accelerate.

In [ ]:
!pip install bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 8.9 MB/s eta 0:00:00


Accelerate é uma biblioteca que permite que o mesmo código PyTorch seja executado em qualquer configuração distribuída adicionando apenas quatro linhas de código. Otimiza as operações do PyTorch, especialmente na GPU.

In [ ]:
# accelerate: treino distribuído, mixed precision, consumer hardware
# peft: métodos para treino eficiente (sem necessidade de tunar todos os parâmetros do modelo)
!pip install accelerate

Instala a interface pytorch para o BERT by Hugging Face.

Fornece uma maneira direta de usar modelos pré-treinados.

In [ ]:
# Instala a última versão da biblioteca
#!pip install transformers

# A última versão do huggingface apresenta um problema:
# UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1`
# https://discuss.huggingface.co/t/help-with-llama-2-finetuning-setup/50035
# Usar a versão 4.31.0

# Instala uma versão específica da biblioteca
!pip install -U transformers==4.31.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 81.7 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.14.1
    Uninstalling tokenizers-0.14.1:
      Successfully uninstalled tokenizers-0.14.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.34.1
    Uninstalling transformers-4.34.1:
      Successfully uninstalled transformers-4.34.1


# 2 - Carregando o LLM



## 2.1 - Nome do modelo de linguagem

Define o nome do modelo a ser carregado
Lista dos modelos:
  - https://huggingface.co/decapoda-research/llama-7b-hf
  - https://huggingface.co/decapoda-research/llama-13b-hf
  - https://huggingface.co/decapoda-research/llama-30b-hf
  - https://huggingface.co/decapoda-research/llama-65b-hf

In [ ]:
nome_modelo = "decapoda-research/llama-7b-hf"
# nome_modelo = "decapoda-research/llama-13b-hf"
# nome_modelo = "decapoda-research/llama-30b-hf"
# nome_modelo = "decapoda-research/llama-65b-hf"

## 2.2 - Carrega o tokenizador

Carregando o **tokenizador** da comunidade.

In [ ]:
# Importando as bibliotecas do Tokenizador
from transformers import LlamaTokenizer

# Carregando o Tokenizador da comunidade
print('Carregando o tokenizador ' + nome_modelo + ' da comunidade...')

tokenizer = LlamaTokenizer.from_pretrained(nome_modelo)

Carregando o tokenizador decapoda-research/llama-7b-hf da comunidade...


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'LLaMATokenizer'. 
The class this function is called from is 'LlamaTokenizer'.
You are using the legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565


## 2.3 - Carregando o LLM

Carregando o **modelo** da comunidade.

Parametrização do from_pretrained
https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-cpu-and-gpu

In [ ]:
# Importando as bibliotecas do Modelo
from transformers import LlamaForCausalLM
from peft import PeftModel
import time

# Guarda o tempo de início do carregamento do modelo
tempo_inicio = time.time()

# Carregando o Modelo da comunidade
print('Carregando o modelo ' + nome_modelo + ' da comunidade...')

model = LlamaForCausalLM.from_pretrained(nome_modelo,
                                             #torch_dtype=torch.float16, #default
                                             load_in_8bit=True,
                                             device_map="auto"
                                             )

model = PeftModel.from_pretrained(model, "dominguesm/alpaca-lora-ptbr-7b")

# Coloca o modelo e modo avaliação
model.eval()

print("Tempo de carregamento do modelo:  {:} (h:mm:ss)".format(formataTempo(time.time() - tempo_inicio)))

Carregando o modelo decapoda-research/llama-7b-hf da comunidade...


Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

Tempo de carregamento do modelo:  0:03:02 (h:mm:ss)


In [ ]:
print(model.config.max_position_embeddings)

2048


In [ ]:
print(model.config)

LlamaConfig {
  "_name_or_path": "decapoda-research/llama-7b-hf",
  "architectures": [
    "LLaMAForCausalLM"
  ],
  "bos_token_id": 0,
  "eos_token_id": 1,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 11008,
  "max_position_embeddings": 2048,
  "max_sequence_length": 2048,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 32,
  "pad_token_id": -1,
  "pretraining_tp": 1,
  "quantization_config": {
    "bnb_4bit_compute_dtype": "float32",
    "bnb_4bit_quant_type": "fp4",
    "bnb_4bit_use_double_quant": false,
    "llm_int8_enable_fp32_cpu_offload": false,
    "llm_int8_has_fp16_weight": false,
    "llm_int8_skip_modules": null,
    "llm_int8_threshold": 6.0,
    "load_in_4bit": false,
    "load_in_8bit": true
  },
  "rms_norm_eps": 1e-06,
  "rope_scaling": null,
  "tie_word_embeddings": false,
  "torch_dtype": "float16",
  "transformers_version": "4.31.0",
  "use_cache": true,

# 3 - Analisando a geração de textos



## 3.1 - Geração de texto


In [ ]:
# Define o documento base
documento = "Como empilhar elementos em uma pilha?"
#documento = "How to push elements in a stack"
#documento = "O comando SQL para extrair todos os usuários cujo nome começa com A é:"
#documento = "Bom dia professor, tudo bem ?"
# documento = "The SQL command to extract all the users whose name starts with A is:"
#documento = "How to push elements in a stack"
#documento = "Write code for finding the prime number in python ?"
# documento = "Escrever código para encontrar o número primo em python?"

# Prepara o prompt para enviar ao modelo realizando sua tokenização
# Se pt for especificado, ele retornará tensores em vez de lista de inteiros python e tokenizará os documentos
input = tokenizer(documento, return_tensors="pt")

# Mostra os tokens com seus índices
i = 0
for tup in input.input_ids[0]:
    # print(tup.item())
    print("{} {}".format(i, tokenizer.convert_ids_to_tokens(tup.item())))
    i= i + 1

0 <unk>
1 ▁Como
2 ▁emp
3 il
4 har
5 ▁elementos
6 ▁em
7 ▁uma
8 ▁pil
9 ha
10 ?


In [ ]:
# Import das bibliotecas
from transformers import GenerationConfig
import torch
import gc

# Configuração da geração
configuracao_geracao = GenerationConfig(
    temperature=0.1,
    top_p=0.75,
    num_beams=4,
)

# Conecta a entrada prerada ao mesmo dispositivo de computação do modelo
input_ids = input["input_ids"].to(model.device)

# Envia a prompt preparado ao modelo
outputs = model.generate(
        input_ids=input_ids,
        generation_config=configuracao_geracao,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=256
    )

# Liberar memória
gc.collect()
torch.cuda.empty_cache()

In [ ]:
print(len(outputs))

4


In [ ]:
print(outputs)

BeamSearchDecoderOnlyOutput(sequences=tensor([[    0, 17295,  3710,   309,  8222, 29290,   953,  3672,  8230,  2350,
         29973,     2,     1]], device='cuda:0'), sequences_scores=tensor([-0.1733], device='cuda:0'), scores=(tensor([[-22.9545, -30.8060,  -2.0795,  ..., -20.8763, -18.7435, -16.8041],
        [-22.9545, -30.8060,  -2.0795,  ..., -20.8763, -18.7435, -16.8041],
        [-22.9545, -30.8060,  -2.0795,  ..., -20.8763, -18.7435, -16.8041],
        [-22.9545, -30.8060,  -2.0795,  ..., -20.8763, -18.7435, -16.8041]],
       device='cuda:0'), tensor([[-2.1860e+01, -3.3891e+01, -6.9576e+00,  ..., -1.6733e+01,
         -1.6320e+01, -1.6253e+01],
        [-2.4099e+01, -2.5034e-06, -2.2079e+01,  ..., -2.4102e+01,
         -2.4607e+01, -2.3211e+01],
        [-2.4931e+01, -3.3615e+01, -1.3068e+01,  ..., -2.3533e+01,
         -2.1175e+01, -2.0650e+01],
        [-2.0389e+01, -2.8709e+01, -9.0643e+00,  ..., -1.8892e+01,
         -1.6359e+01, -1.8967e+01]], device='cuda:0'), tensor([[-2

Decodificação

Nossa etapa de geração gera uma matriz de tokens em vez de palavras. Para converter esses tokens em palavras, precisamos realizar sua decodificação.

In [ ]:
# Mostra o resultado
for s in outputs.sequences:
  # Decodifica a saída
  # skip_special_tokens=True retira os tokens especiais da saída da decodificação
  output = tokenizer.decode(s, skip_special_tokens=True)
  print(output)

Como empilhar elementos em uma pilha?


In [ ]:
# Mostra o resultado
# skip_special_tokens=True retira os tokens especiais da saída da decodificação
print(tokenizer.decode(outputs.sequences[0], skip_special_tokens=True))

Como empilhar elementos em uma pilha?


# 4 - Exemplos de textos emparelhados

In [ ]:
def gerar_prompt(texto, entrada=None):
    if entrada:
        return f"""Abaixo está uma instrução que descreve uma tarefa, emparelhada com uma entrada que fornece mais contexto. Escreva uma resposta que conclua adequadamente a solicitação.

### Instruções:
{texto}

### Entrada:
{entrada}

### Resposta:"""
    else:
        return f"""Abaixo está uma instrução que descreve uma tarefa. Escreva uma resposta que conclua adequadamente a solicitação.

### Instruções:
{texto}

### Resposta:"""

In [ ]:
# Import das bibliotecas
from transformers import GenerationConfig
import gc

# Configuração da geração
configuracao_geracao = GenerationConfig(
    temperature=0.1,
)

def avaliarTexto(texto, entrada=None):
    # Recupera o prompt
    prompt = gerar_prompt(texto, entrada)

    # Prepara o prompt para enviar ao modelo realizando sua tokenização
    inputs = tokenizer(prompt, return_tensors="pt")
    # Conecta a entrada prerada ao mesmo dispositivo de computação do modelo
    input_ids = inputs["input_ids"].to(model.device)

    # Envia a prompt preparado ao modelo
    output = model.generate(
        input_ids=input_ids,
        generation_config=configuracao_geracao,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=256
    )

    # Mostra a saída
    for s in output.sequences:
        # Decodifica a saída
        output = tokenizer.decode(s, skip_special_tokens=True)
        # Recupera a saída da resposta
        saida = output.split("### Resposta:")[1].strip()
        print("Resposta: \n" + saida)

    # Liberar memória
    del input_ids
    del output
    gc.collect()
    torch.cuda.empty_cache()

## 4.1 - Tarefa simples

In [ ]:
texto = 'Me fale sobre algoritmos.'
avaliarTexto(texto)

Resposta: 
Os algoritmos são um tipo de programa que pode ser usado para resolver problemas complexos. Eles são usados para automatizar tarefas, como processamento de linguagem natural, reconhecimento de imagem e processamento de dados. Os algoritmos podem ser usados para resolver problemas de várias maneiras, como analisar dados, encontrar padrões e fazer previsões. Os algoritmos podem ser usados para resolver problemas de várias maneiras, como analisar dados, encontrar padrões e fazer previsões. Os algoritmos podem ser usados para resolver problemas de várias maneiras, como analisar dados, encontrar padrões e fazer previsões. Os algoritmos podem ser usados para resolver problemas de várias maneiras, como analisar dados, encontrar padrões e fazer previsões. Os algoritmos podem ser usados para resolver problemas de várias maneiras, como analisar dados, encontrar padrões e fazer previsões


## 4.2 - Tarefa com entrada

In [ ]:
texto = 'Dada a fórmula química, calcule a massa molar.'

entrada = 'CaCl2'

avaliarTexto(texto, entrada)

Resposta: 
120,01 g/mol


In [ ]:
texto = 'Faça quatro perguntas sobre a seguinte passagem:'

entrada = 'A anatomia de uma abelha é bastante intrincada. Tem três partes do corpo: a cabeça, o tórax e o abdômen. A cabeça consiste em órgãos sensoriais, três olhos simples e dois olhos compostos e vários apêndices. O tórax tem três pares de pernas e dois pares de asas, enquanto o abdômen contém a maioria dos órgãos da abelha, incluindo o sistema reprodutivo e o sistema digestivo.'

avaliarTexto(texto, entrada)

Resposta: 
1. Quais são os três tipos de olhos que a abelha tem? 2. Quais são os dois tipos de olhos que a abelha tem? 3. Quais são os três tipos de pernas que a abelha tem? 4. Quais são os dois tipos de pernas que a abelha tem?


In [ ]:
texto = 'Analise o documento jurídico fornecido e explique os pontos-chave.'

entrada = 'O seguinte é um trecho de um contrato entre duas partes, rotulado como "Empresa A" e "Empresa B": "A Empresa A concorda em fornecer assistência razoável à Empresa B para garantir a precisão das demonstrações financeiras que fornece. Isso inclui permitir à Empresa um acesso razoável ao pessoal e outros documentos que possam ser necessários para a revisão da Empresa B. A Empresa B concorda em manter o documento fornecido pela Empresa A em confiança e não divulgará as informações a terceiros sem a permissão explícita da Empresa A".'

avaliarTexto(texto, entrada)

Resposta: 
O contrato entre as duas partes, rotulado como "Empresa A" e "Empresa B", é um acordo para fornecer assistência razoável à Empresa B para garantir a precisão das demonstrações financeiras que fornece. Isso inclui permitir à Empresa um acesso razoável ao pessoal e outros documentos que possam ser necessários para a revisão da Empresa B. A Empresa B concorda em manter o documento fornecido pela Empresa A em confiança e não divulgará as informações a terceiros sem a permissão explícita da Empresa A.


# 5 - Exemplos de injeção de padrões em prompts

 A injeção de padrões faz ignora filtros ou manipula o LLM usando prompts cuidadosamente elaborados que fazem o modelo ignorar instruções anteriores ou executar ações não intencionais.

 https://medium.com/@austin-stubbs/llm-security-types-of-prompt-injection-d7ad8d7d75a3


### 5.1 - Extração de Informação

In [ ]:
def gerar_promptEI(texto):
    ### texto:
    return f"""TEXTO: {texto}
Dado o texto acima, extraia informações importantes no formato abaixo:
<CHAVE>:<VALOR>
### Resposta: """

In [ ]:
# Import das bibliotecas
from transformers import GenerationConfig
import torch
import gc

# Configuração da geração
configuracao_geracao = GenerationConfig(
    temperature=0.1,
    top_p=0.75,
    num_beams=4,
)

def avaliarEI(texto):
    # Recupera o prompt
    prompt = gerar_promptEI(texto)

    # Prepara o prompt para enviar ao modelo realizando sua tokenização
    inputs = tokenizer(prompt, return_tensors="pt")
    # Conecta a entrada prerada ao mesmo dispositivo de computação do modelo
    input_ids = inputs["input_ids"].to(model.device)

    # Envia a prompt preparado ao modelo
    output = model.generate(
        input_ids=input_ids,
        generation_config=configuracao_geracao,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=512
    )

    # Mostra a saída
    for s in output.sequences:
        # Decodifica a saída
        output = tokenizer.decode(s, skip_special_tokens=True)
        # Recupera a saída da resposta
        saida = output.split("### Resposta:")[1].strip()
        print("Resposta: \n" + saida)

    # Liberar memória
    del input_ids
    del output
    gc.collect()
    torch.cuda.empty_cache()

In [ ]:
texto = "Alan Mathison Turing (Londres, 23 de junho de 1912 — Wilmslow, Cheshire, 7 de junho de 1954)"\
        "foi um matemático, cientista da computação, lógico, criptoanalista, filósofo e biólogo teórico "\
        "britânico. Turing foi altamente influente no desenvolvimento da moderna ciência da computação "\
        "teórica, proporcionando uma formalização dos conceitos de algoritmo e computação com a máquina "\
        "de Turing, que pode ser considerada um modelo de um computador de uso geral. Ele é amplamente "\
        "considerado o pai da ciência da computação teórica e da inteligência artificial. Apesar dessas "\
        "realizações ele nunca foi totalmente reconhecido em seu país de origem durante sua vida por ser "\
        "homossexual e porque grande parte de seu trabalho foi coberto pela Lei de Segredos Oficiais."

avaliarEI(texto)

Resposta: 
1


### 5.2 - Entidade Nomeada

In [ ]:
def gerar_promptEN(texto):
    ### texto:
    return f"""Detecte as entidades nomeadas no texto a seguir delimitado por aspas triplas.
Retorne a resposta no formato json com spans( Um array que representa o intervalo de caracteres (índices) nos quais a entidade nomeada ocorre no texto original. O primeiro valor no array é o índice inicial e o segundo é o índice final) das entidades nomeadas com os campos \'entidadeNomeada\', \'tipo\', \'span\'.
Retorne todas as entidades
'''{texto}'''
arquivo no formato json:
### Resposta: """

In [ ]:
# Import das bibliotecas
from transformers import GenerationConfig
import torch
import gc

# Configuração da geração
configuracao_geracao = GenerationConfig(
    temperature=0.1,
    top_p=0.75,
    num_beams=4,
)

def avaliarEN(texto):
    # Recupera o prompt
    prompt = gerar_promptEN(texto)

    # Prepara o prompt para enviar ao modelo realizando sua tokenização
    inputs = tokenizer(prompt, return_tensors="pt")
    # Conecta a entrada prerada ao mesmo dispositivo de computação do modelo
    input_ids = inputs["input_ids"].to(model.device)

    # Envia a prompt preparado ao modelo
    output = model.generate(
        input_ids=input_ids,
        generation_config=configuracao_geracao,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=512
    )

    # Mostra a saída
    for s in output.sequences:
        # Decodifica a saída
        output = tokenizer.decode(s, skip_special_tokens=True)
        # Recupera a saída da resposta
        saida = output.split("### Resposta:")[1].strip()
        print("Resposta: \n" + saida)

    # Liberar memória
    del input_ids
    del output
    gc.collect()
    torch.cuda.empty_cache()

In [ ]:
texto = "Alan Mathison Turing (Londres, 23 de junho de 1912 — Wilmslow, Cheshire, 7 de junho de 1954)"\
        "foi um matemático, cientista da computação, lógico, criptoanalista, filósofo e biólogo teórico "\
        "britânico. Turing foi altamente influente no desenvolvimento da moderna ciência da computação "\
        "teórica, proporcionando uma formalização dos conceitos de algoritmo e computação com a máquina "\
        "de Turing, que pode ser considerada um modelo de um computador de uso geral. Ele é amplamente "\
        "considerado o pai da ciência da computação teórica e da inteligência artificial. Apesar dessas "\
        "realizações ele nunca foi totalmente reconhecido em seu país de origem durante sua vida por ser "\
        "homossexual e porque grande parte de seu trabalho foi coberto pela Lei de Segredos Oficiais."

avaliarEN(texto)

Resposta: 
1


## 5.3 - Análise de sentimentos

### 5.3.1 - Análise de sentimentos 1

In [ ]:
def gerar_promptAS1(texto):

  return f"""Classifique os exemplos a seguir de acordo com as seguintes polaridades Positivo, Negativo e Neutro.
EXEMPLO:\n {texto}
### Resposta:"""

In [ ]:
# Import das bibliotecas
from transformers import GenerationConfig
import torch
import gc

# Configuração da geração
configuracao_geracao = GenerationConfig(
    temperature=0.1,
    top_p=0.75,
    num_beams=4,
)

def avaliarAS1(texto):
    # Recupera o prompt
    prompt = gerar_promptAS1(texto)

    # Prepara o prompt para enviar ao modelo realizando sua tokenização
    inputs = tokenizer(prompt, return_tensors="pt")
    # Conecta a entrada prerada ao mesmo dispositivo de computação do modelo
    input_ids = inputs["input_ids"].to(model.device)

    # Envia a prompt preparado ao modelo
    output = model.generate(
        input_ids=input_ids,
        generation_config=configuracao_geracao,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=256
    )

    # Mostra a saída
    for s in output.sequences:
        # Decodifica a saída
        output = tokenizer.decode(s, skip_special_tokens=True)
        # Recupera a saída da resposta
        saida = output.split("### Resposta:")[1].strip()
        print("Resposta: \n" + saida)

    # Liberar memória
    del input_ids
    del output
    gc.collect()
    torch.cuda.empty_cache()

In [ ]:
texto = "1 - Minha Experiência na loja foi incrível."\
        "2 - Eu acho que podiam melhorar o produto."\
        "3 - O atendimento foi horrível!"\
        "4 - Não volto mais."\
        "5 - Recomendo demais a banoffe. É uma delícia!"

avaliarAS1(texto)

Resposta: 
1 - Minha Experiência na loja foi incrível.2 - Eu acho que podiam melhorar o produto.3 - O atendimento foi horrível!4 - Não volto mais.5 - Recomendo demais a banoffe. É uma delícia!


### 5.3.2 - Análise de sentimentos 2

In [ ]:
def gerar_promptAS2(texto):

  return  f"""EXEMPLO: {texto}
Classifique os exemplos de declarações acima de acordo com as polaridades Positivo, Negativo e Neutro.
Utilize o seguinte formato:\n###DECLARAÇÃO:<DECLARAÇÃO>\n###POLARIDADE:<POLARIDADE>.
### Resposta:"""

In [ ]:
# Import das bibliotecas
from transformers import GenerationConfig
import torch
import gc

# Configuração da geração
configuracao_geracao = GenerationConfig(
    temperature=0.1,
    top_p=0.75,
    num_beams=4,
)

def avaliarAS2(texto):
    # Recupera o prompt
    prompt = gerar_promptAS2(texto)

    # Prepara o prompt para enviar ao modelo realizando sua tokenização
    inputs = tokenizer(prompt, return_tensors="pt")
    # Conecta a entrada prerada ao mesmo dispositivo de computação do modelo
    input_ids = inputs["input_ids"].to(model.device)

    # Envia a prompt preparado ao modelo
    output = model.generate(
        input_ids=input_ids,
        generation_config=configuracao_geracao,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=512
    )

    # Mostra a saída
    for s in output.sequences:
        # Decodifica a saída
        output = tokenizer.decode(s, skip_special_tokens=True)
        # Recupera a saída da resposta
        saida = output.split("### Resposta:")[1].strip()
        print("Resposta: \n" + saida)

    # Liberar memória
    del input_ids
    del output
    gc.collect()
    torch.cuda.empty_cache()

In [ ]:
texto = "1 - Minha Experiência na loja foi incrível."\
        "2 - Eu acho que podiam melhorar o produto."\
        "3 - O atendimento foi horrível!"\
        "4 - Não volto mais."\
        "5 - Recomendo demais a banoffe. É uma delícia!"

avaliarAS2(texto)

Resposta: 
###DECLARAÇÃO:<DECLARAÇÃO>
###POLARIDADE:<POLARIDADE>.


## 5.5 - Pergunta e resposta

In [ ]:
def gerar_promptPR(texto):
    '''
      Alterações no texto e tabulação impedem a geração da resposta.
    '''
    return f"""Dado o texto a seguir: {texto}\n
            Gere quatro questões em língua portuguesa e suas respectivas respostas utilizando o template abaixo.\n
            Preserve a exata formatação do template apresentado: \n
            PERGUNTA:<PERGUNTA>
            RESPOSTA:<RESPOSTA>
            ### Resposta:"""

In [ ]:
# Import das bibliotecas
from transformers import GenerationConfig
import torch
import gc

# Configuração da geração
configuracao_geracao = GenerationConfig(
    temperature=0.1,
    top_p=0.75,
    num_beams=4,
)

def avaliarPR(texto):
    # Recupera o prompt
    prompt = gerar_promptPR(texto)

    # Prepara o prompt para enviar ao modelo realizando sua tokenização
    inputs = tokenizer(prompt, return_tensors="pt")
    # Conecta a entrada prerada ao mesmo dispositivo de computação do modelo
    input_ids = inputs["input_ids"].to(model.device)

    # Envia a prompt preparado ao modelo
    output = model.generate(
        input_ids=input_ids,
        generation_config=configuracao_geracao,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=512
    )

    # Mostra a saída
    for s in output.sequences:
        # Decodifica a saída
        output = tokenizer.decode(s, skip_special_tokens=True)
        # Recupera a saída da resposta
        saida = output.split("### Resposta:")[1].strip()
        print("Resposta: \n" + saida)

    # Liberar memória
    del input_ids
    del output
    gc.collect()
    torch.cuda.empty_cache()

In [ ]:
texto = "Alan Mathison Turing (Londres, 23 de junho de 1912 — Wilmslow, Cheshire, 7 de junho de 1954)"\
        "foi um matemático, cientista da computação, lógico, criptoanalista, filósofo e biólogo teórico "\
        "britânico. Turing foi altamente influente no desenvolvimento da moderna ciência da computação "\
        "teórica, proporcionando uma formalização dos conceitos de algoritmo e computação com a máquina "\
        "de Turing, que pode ser considerada um modelo de um computador de uso geral. Ele é amplamente "\
        "considerado o pai da ciência da computação teórica e da inteligência artificial. Apesar dessas "\
        "realizações ele nunca foi totalmente reconhecido em seu país de origem durante sua vida por ser "\
        "homossexual e porque grande parte de seu trabalho foi coberto pela Lei de Segredos Oficiais."

# Tempo de processamento: ~2m20s
avaliarPR(texto)

Resposta: 
PERGUNTA: Qual é o nome de sua cidade natal?
            RESPOSTA: 

            PERGUNTA: Qual é o nome de sua cidade natal?
            RESPOSTA: 

            PERGUNTA: Qual é o nome de sua cidade natal?
            RESPOSTA: 

            PERGUNTA: Qual é o nome de sua cidade natal?
            RESPOSTA: 

            PERGUNTA: Qual é o nome de sua cidade natal?
            RESPOSTA: 

            PERGUNTA: Qual é o nome de sua cidade natal?
            RESPOSTA: 

            PERGUNTA: Qual é o nome de sua cidade natal?
            RESPOSTA: 

            PERGUNTA: Qual é o nome de sua cidade natal?
            RESPOSTA: 

            PERGUNTA: Qual é o nome de sua cidade natal?
            RESPOSTA: 

            PERGUNTA: Qual é o nome de sua cidade natal?
            RESPOSTA: 

            PERGUNTA: Qual é o nome de sua cidade natal?
            RESPOSTA: 

            PERGUNTA: Qual é o nome de sua cidade natal?
            RESPOSTA: 

            PERGUNTA: Qual é 

# 6 - Exemplos de padrão de pessoa (padrão persona) em prompts

## 6.1 Um matemático

In [ ]:
def gerar_prompt(texto):

    return  f"""{texto}\n
### Resposta:"""

In [ ]:
# Import das bibliotecas
from transformers import GenerationConfig
import gc

# Configuração da geração
configuracao_geracao = GenerationConfig(
    temperature=0.1,
)

def avaliarTexto(texto):
    # Recupera o prompt
    prompt = gerar_prompt(texto)

    # Prepara o prompt para enviar ao modelo realizando sua tokenização
    inputs = tokenizer(prompt, return_tensors="pt")
    # Conecta a entrada prerada ao mesmo dispositivo de computação do modelo
    input_ids = inputs["input_ids"].to(model.device)

    # Envia a prompt preparado ao modelo
    output = model.generate(
        input_ids=input_ids,
        generation_config=configuracao_geracao,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=512
    )

    # Mostra a saída
    for s in output.sequences:
        # Decodifica a saída
        output = tokenizer.decode(s, skip_special_tokens=True)
        # Recupera a saída da resposta
        saida = output.split("### Resposta:")[1].strip()
        print("Resposta: \n" + saida)

    # Liberar memória
    del input_ids
    del output
    gc.collect()
    torch.cuda.empty_cache()

In [ ]:
texto = 'Escreva como se fosse um professor de matemática. Me explique no idioma portuguesa a importância do teorema de pitágoras.'

avaliarTexto(texto)

Resposta: 
O teorema de Pitágoras afirma que a soma dos quadrados dos lados de um triângulo é igual ao quadrado da soma dos ângulos.


## 6.2 Um advogado

In [ ]:
def gerar_prompt(texto):

    return  f"""{texto}\n
### Resposta:"""

In [ ]:
# Import das bibliotecas
from transformers import GenerationConfig
import gc

# Configuração da geração
configuracao_geracao = GenerationConfig(
    temperature=0.1,
)

def avaliarTexto(texto):
    # Recupera o prompt
    prompt = gerar_prompt(texto)

    # Prepara o prompt para enviar ao modelo realizando sua tokenização
    inputs = tokenizer(prompt, return_tensors="pt")
    # Conecta a entrada prerada ao mesmo dispositivo de computação do modelo
    input_ids = inputs["input_ids"].to(model.device)

    # Envia a prompt preparado ao modelo
    output = model.generate(
        input_ids=input_ids,
        generation_config=configuracao_geracao,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=512
    )

    # Mostra a saída
    for s in output.sequences:
        # Decodifica a saída
        output = tokenizer.decode(s, skip_special_tokens=True)
        # Recupera a saída da resposta
        saida = output.split("### Resposta:")[1].strip()
        print("Resposta: \n" + saida)

    # Liberar memória
    del input_ids
    del output
    gc.collect()
    torch.cuda.empty_cache()

In [ ]:
texto = 'Escreva como se fosse um advogado brasileiro especialista em direito penal. '\
        'Pontue de forma resumida as possíveis penas para um caso de lesão corporal leve sem contexto de violência doméstica.'

avaliarTexto(texto)

Resposta: 
Escreva: "O que você está sugerindo é que o acusado deve ser condenado a uma pena de prisão de 1 a 3 anos, dependendo da gravidade do crime. Isso é uma pena muito severa para um crime de lesão corporal leve sem contexto de violência doméstica. Eu recomendo que o acusado seja condenado a uma pena de prisão de 1 a 6 meses, dependendo da gravidade do crime."


## 6.3 Um astrofísico

In [ ]:
def gerar_prompt(texto):

    return  f"""{texto}\n
### Resposta:"""

In [ ]:
# Import das bibliotecas
from transformers import GenerationConfig
import gc

# Configuração da geração
configuracao_geracao = GenerationConfig(
    temperature=0.1,
)

def avaliarTexto(texto):
    # Recupera o prompt
    prompt = gerar_prompt(texto)

    # Prepara o prompt para enviar ao modelo realizando sua tokenização
    inputs = tokenizer(prompt, return_tensors="pt")
    # Conecta a entrada prerada ao mesmo dispositivo de computação do modelo
    input_ids = inputs["input_ids"].to(model.device)

    # Envia a prompt preparado ao modelo
    output = model.generate(
        input_ids=input_ids,
        generation_config=configuracao_geracao,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=512
    )

    # Mostra a saída
    for s in output.sequences:
        # Decodifica a saída
        output = tokenizer.decode(s, skip_special_tokens=True)
        # Recupera a saída da resposta
        saida = output.split("### Resposta:")[1].strip()
        print("Resposta: \n" + saida)

    # Liberar memória
    del input_ids
    del output
    gc.collect()
    torch.cuda.empty_cache()

In [ ]:
texto = 'Escreva como se fosse um astrofísico. Me explique por que o universo está expandindo usando o idioma português.'

avaliarTexto(texto)

Resposta: 
O universo está expandindo porque a força de gravidade é a força que mantém os objetos juntos. Quando um objeto é lançado, a força de gravidade diminui e o objeto se afasta do centro da Terra.
